# 쿠팡 데이터 탐색적 데이터 분석 (EDA)

쿠팡 상품, 판매자, 리뷰 및 질문 데이터를 분석하여 데이터의 분포와 특징을 파악합니다.

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from src.database import load_table

## 1. 데이터 로드

Supabase에서 주요 데이터들을 불러옵니다.

In [ ]:
products_df = load_table('products')
sellers_df = load_table('sellers')
reviews_df = load_table('reviews')
questions_df = load_table('questions')

print(f"상품 데이터: {products_df.shape}")
print(f"판매자 데이터: {sellers_df.shape}")
print(f"리뷰 데이터: {reviews_df.shape}")
print(f"질문 데이터: {questions_df.shape}")

## 2. 데이터 기본 정보 확인

각 데이터프레임의 컬럼 정보와 누락된 값(Null)을 확인합니다.

In [ ]:
print("--- 상품 데이터 정보 ---")
print(products_df.info())

# 결측치 있는 데이터 없음
print("\n--- 상품 결측치 분포 ---")
print(products_df.isnull().sum())

In [ ]:

print("\n--- 판매자 데이터 정보 ---")
print(sellers_df.info())
print("\n--- 판매자 결측치 분포 ---")
print(sellers_df.isnull().sum())

# 쿠팡에서 판매자샵 접근을 막은 판매자들은 판매자 ID가 없음
print("\n--- 결측치가 포함된 데이터 ---")
print(sellers_df[sellers_df.isnull().any(axis=1)])

In [ ]:

print("\n--- 리뷰 데이터 정보 ---")
print(reviews_df.info())

# 리뷰에 평점만 있고 제목이나 내용이 없는 데이터
print("\n--- 리뷰 결측치 분포 ---")
print(reviews_df.isnull().sum())

In [ ]:

print("\n--- 문의 데이터 정보 ---")
print(questions_df.info())
print("\n--- 문의 결측치 분포 ---")
print(questions_df.isnull().sum())

# 문의에 대한 답변이 없는 케이스
print("\n--- 결측치가 포함된 데이터 ---")
print(questions_df[questions_df.isnull().any(axis=1)])

## 3. 주요 지표 분포 분석

### 3.1 가격 분포

In [ ]:
fig = go.Figure(data=[go.Histogram(x=products_df['price'], nbinsx=50, marker_color='#636EFA')])
fig.update_layout(
    title='상품 가격 분포',
    xaxis_title='가격',
    yaxis_title='빈도',
    template='plotly_white'
)
fig.show()

### 3.2 평점 및 리뷰 수 분포

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('상품 평점 분포', '리뷰 수 분포'))

fig.add_trace(
    go.Histogram(x=products_df['product_rating'], nbinsx=10, name='평점', marker_color='#EF553B'),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=products_df['review_count'], nbinsx=50, name='리뷰 수', marker_color='#00CC96'),
    row=1, col=2
)

fig.update_layout(height=500, title_text="평점 및 리뷰 수 종합 분석", template='plotly_white')
fig.show()

## 4. 판매자 분석

판매자의 만족도 점수와 등록된 상품 수를 분석합니다.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=sellers_df['satisfaction_score'], nbinsx=20, marker_color='#AB63FA'))
fig.update_layout(
    title='판매자 만족도 점수 분포',
    xaxis_title='만족도',
    yaxis_title='빈도',
    template='plotly_white'
)
fig.show()

## 5. 상관관계 분석

수치형 변수들 간의 상관관계를 확인합니다.

In [ ]:
numeric_df = products_df.select_dtypes(include=['float64', 'int64'])
corr = numeric_df.corr()

# 컬럼명 한글 매핑
column_mapping = {
    'product_id': '상품ID',
    'item_id': '아이템ID',
    'review_count': '리뷰 수',
    'price': '가격',
    'original_price': '정가',
    'candidate_price': '후보가격',
    'discount_rate': '할인율',
    'product_rating': '평점',
    'shipping_fee': '배송비',
    'shipping_days': '배송일수',
    'inquiry_count': '문의수',
    'rating_1': '평점 1점 비율',
    'rating_2': '평점 2점 비율',
    'rating_3': '평점 3점 비율',
    'rating_4': '평점 4점 비율',
    'rating_5': '평점 5점 비율',
}

fig = go.Figure(data=go.Heatmap(
    z=corr.values,
    x=[column_mapping.get(col, col) for col in corr.columns],
    y=[column_mapping.get(col, col) for col in corr.index],
    colorscale='RdBu_r',
    zmin=-1, zmax=1,
    text=corr.values.round(2),
    texttemplate='%{text}',
    textfont=dict(size=10)
))

fig.update_layout(
    title='상태 지표 간 상관관계 히트맵',
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    template='plotly_white'
)
fig.show()

## 6. 어뷰징 판매자 분석

정상 판매자와 어뷰징 판매자의 특성을 비교 분석합니다.

In [ ]:
# 어뷰징 vs 정상 판매자 비교
abusing_sellers = sellers_df[sellers_df['is_abusing_seller']]
normal_sellers = sellers_df[~sellers_df['is_abusing_seller']]

print(f"어뷰징 판매자: {len(abusing_sellers)}개 ({len(abusing_sellers)/len(sellers_df)*100:.1f}%)")
print(f"정상 판매자: {len(normal_sellers)}개 ({len(normal_sellers)/len(sellers_df)*100:.1f}%)")

# 주요 지표 비교
fig = make_subplots(rows=1, cols=3, subplot_titles=('만족도 점수', '리뷰 수', '등록 상품 수'))

for i, (col, title) in enumerate([('satisfaction_score', '만족도'), ('review_count', '리뷰 수'), ('total_product_count', '상품 수')], 1):
    fig.add_trace(go.Box(y=normal_sellers[col], name='정상', marker_color='#00CC96'), row=1, col=i)
    fig.add_trace(go.Box(y=abusing_sellers[col], name='어뷰징', marker_color='#EF553B'), row=1, col=i)

fig.update_layout(height=400, title_text="어뷰징 vs 정상 판매자 지표 비교", showlegend=False, template='plotly_white')
fig.show()

## 7. 카테고리 분석

상품의 카테고리 분포와 카테고리별 특성을 분석합니다.

In [ ]:
# 1차 카테고리 추출 (첫 번째 > 이전 부분)
products_df['category_1st'] = products_df['category'].apply(
    lambda x: x.split(' > ')[0] if pd.notna(x) and ' > ' in str(x) else x
)

# 카테고리별 상품 수
category_counts = products_df['category_1st'].value_counts().head(15)

fig = go.Figure(data=[
    go.Bar(x=category_counts.values, y=category_counts.index, orientation='h', marker_color='#636EFA')
])
fig.update_layout(
    title='1차 카테고리별 상품 수 (Top 15)',
    xaxis_title='상품 수',
    yaxis_title='카테고리',
    height=500,
    template='plotly_white',
    yaxis={'categoryorder': 'total ascending'}
)
fig.show()

In [ ]:
# 카테고리별 평균 가격 및 평점
category_stats = products_df.groupby('category_1st').agg({
    'price': 'mean',
    'product_rating': 'mean',
    'review_count': 'sum'
}).round(2)

top_categories = category_counts.head(10).index.tolist()
category_stats_top = category_stats.loc[category_stats.index.isin(top_categories)]

fig = make_subplots(rows=1, cols=2, subplot_titles=('카테고리별 평균 가격', '카테고리별 평균 평점'))

fig.add_trace(
    go.Bar(x=category_stats_top['price'], y=category_stats_top.index, orientation='h', 
           name='평균 가격', marker_color='#636EFA'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=category_stats_top['product_rating'], y=category_stats_top.index, orientation='h',
           name='평균 평점', marker_color='#EF553B'),
    row=1, col=2
)

fig.update_layout(height=500, title_text="카테고리별 가격 및 평점 분석", template='plotly_white', showlegend=False)
fig.show()

## 8. 리뷰 심층 분석

리뷰 평점 분포, 텍스트 길이, 시간대별 분포를 분석합니다.

In [ ]:
# 리뷰 평점 분포
rating_counts = reviews_df['review_rating'].value_counts().sort_index()

fig = go.Figure(data=[
    go.Bar(x=rating_counts.index.astype(str), y=rating_counts.values, 
           marker_color=['#EF553B', '#FFA15A', '#FECB52', '#00CC96', '#636EFA'])
])
fig.update_layout(
    title='리뷰 평점 분포',
    xaxis_title='평점',
    yaxis_title='리뷰 수',
    template='plotly_white'
)
fig.show()

# 평점별 통계
print("\n=== 평점별 리뷰 수 ===")
for rating in sorted(reviews_df['review_rating'].unique()):
    count = len(reviews_df[reviews_df['review_rating'] == rating])
    pct = count / len(reviews_df) * 100
    print(f"  {rating}점: {count}개 ({pct:.1f}%)")

In [ ]:
# 리뷰 텍스트 길이 분석
reviews_df['text_length'] = reviews_df['review_text'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)

fig = make_subplots(rows=1, cols=2, subplot_titles=('리뷰 텍스트 길이 분포', '평점별 평균 텍스트 길이'))

fig.add_trace(
    go.Histogram(x=reviews_df['text_length'], nbinsx=30, marker_color='#636EFA'),
    row=1, col=1
)

avg_length_by_rating = reviews_df.groupby('review_rating')['text_length'].mean()
fig.add_trace(
    go.Bar(x=avg_length_by_rating.index.astype(str), y=avg_length_by_rating.values, marker_color='#EF553B'),
    row=1, col=2
)

fig.update_layout(height=400, title_text="리뷰 텍스트 길이 분석", template='plotly_white', showlegend=False)
fig.show()

print(f"\n평균 리뷰 길이: {reviews_df['text_length'].mean():.1f}자")
print(f"최대 리뷰 길이: {reviews_df['text_length'].max()}자")

## 9. 질문-답변 분석

상품 문의의 답변율과 답변 시간을 분석합니다.

In [ ]:
# 답변 여부 분석
questions_df['has_answer'] = questions_df['answer'].apply(lambda x: pd.notna(x) and str(x).strip() != '')

answered = questions_df['has_answer'].sum()
unanswered = len(questions_df) - answered

print(f"총 질문 수: {len(questions_df)}개")
print(f"답변 완료: {answered}개 ({answered/len(questions_df)*100:.1f}%)")
print(f"미답변: {unanswered}개 ({unanswered/len(questions_df)*100:.1f}%)")

# 파이 차트
fig = go.Figure(data=[go.Pie(
    labels=['답변 완료', '미답변'],
    values=[answered, unanswered],
    marker_colors=['#00CC96', '#EF553B'],
    hole=0.4
)])
fig.update_layout(title='질문 답변율', template='plotly_white')
fig.show()

## 10. 가격 이상치 및 할인율 분석

비정상적인 가격이나 할인율을 가진 상품을 분석합니다.

In [ ]:
# 가격 통계
print("=== 가격 통계 ===")
print(products_df['price'].describe())

# 가격 범위별 분포
price_bins = [0, 10000, 50000, 100000, 500000, float('inf')]
price_labels = ['~1만원', '1~5만원', '5~10만원', '10~50만원', '50만원+']
products_df['price_range'] = pd.cut(products_df['price'], bins=price_bins, labels=price_labels)

price_range_counts = products_df['price_range'].value_counts()

fig = go.Figure(data=[go.Pie(
    labels=price_range_counts.index.tolist(),
    values=price_range_counts.values,
    hole=0.3
)])
fig.update_layout(title='가격대별 상품 분포', template='plotly_white')
fig.show()

# 할인율 분포
fig = go.Figure(data=[go.Histogram(x=products_df['discount_rate'], nbinsx=20, marker_color='#FFA15A')])
fig.update_layout(
    title='할인율 분포',
    xaxis_title='할인율 (%)',
    yaxis_title='상품 수',
    template='plotly_white'
)
fig.show()

## 11. 판매자별 상품 분포

판매자당 등록된 상품 수 분포를 분석합니다.

In [ ]:
# 판매자별 상품 수 (데이터 내)
vendor_product_counts = products_df['vendor_name'].value_counts()

print(f"총 판매자 수: {len(vendor_product_counts)}개")
print(f"판매자당 평균 상품 수: {vendor_product_counts.mean():.2f}개")
print("\n=== 상품이 많은 판매자 Top 10 ===")
print(vendor_product_counts.head(10))

# 분포 시각화
fig = go.Figure(data=[go.Histogram(x=vendor_product_counts.values, nbinsx=20, marker_color='#AB63FA')])
fig.update_layout(
    title='판매자별 상품 수 분포',
    xaxis_title='상품 수',
    yaxis_title='판매자 수',
    template='plotly_white'
)
fig.show()

## 12. 데이터 요약

전체 데이터의 주요 통계를 요약합니다.

In [ ]:
# 종합 요약 테이블
summary_data = {
    '지표': [
        '총 상품 수',
        '총 판매자 수',
        '총 리뷰 수',
        '총 질문 수',
        '어뷰징 판매자 비율',
        '평균 상품 가격',
        '평균 상품 평점',
        '평균 리뷰 길이',
        '질문 답변율',
        '주요 카테고리'
    ],
    '값': [
        f"{len(products_df):,}개",
        f"{len(sellers_df):,}개",
        f"{len(reviews_df):,}개",
        f"{len(questions_df):,}개",
        f"{len(abusing_sellers)/len(sellers_df)*100:.1f}%",
        f"{products_df['price'].mean():,.0f}원",
        f"{products_df['product_rating'].mean():.2f}점",
        f"{reviews_df['text_length'].mean():.0f}자",
        f"{answered/len(questions_df)*100:.1f}%",
        category_counts.index[0]
    ]
}

summary_df = pd.DataFrame(summary_data)
print("=" * 50)
print("데이터 종합 요약")
print("=" * 50)
print(summary_df.to_string(index=False))

## 13. 정상 판매자 vs 어뷰징 판매자: 리뷰 및 질문 비교 분석

seller, product, review, question 테이블을 조인하여 정상 판매자와 어뷰징 판매자 간의 리뷰 및 질문 패턴 차이를 분석합니다.

In [ ]:
# 데이터 조인: seller - product - review - question
# products.vendor_name과 sellers.company_name으로 조인

# 1. 판매자-상품 조인
products_sellers = products_df.merge(
    sellers_df[['company_name', 'is_abusing_seller']], 
    left_on='vendor_name', 
    right_on='company_name', 
    how='left'
)

# 2. 상품-리뷰 조인
reviews_with_seller = reviews_df.merge(
    products_sellers[['product_id', 'is_abusing_seller']], 
    on='product_id', 
    how='left'
)

# 3. 상품-질문 조인
questions_with_seller = questions_df.merge(
    products_sellers[['product_id', 'is_abusing_seller']], 
    on='product_id', 
    how='left'
)

print("=== 조인 결과 ===")
print(f"판매자-상품 조인: {len(products_sellers)}개 상품")
print(f"리뷰 데이터 (판매자 정보 포함): {len(reviews_with_seller)}개")
print(f"질문 데이터 (판매자 정보 포함): {len(questions_with_seller)}개")

# 어뷰징/정상 판매자별 리뷰 및 질문 수
abusing_reviews = reviews_with_seller[reviews_with_seller['is_abusing_seller'] == True]
normal_reviews = reviews_with_seller[reviews_with_seller['is_abusing_seller'] == False]
abusing_questions = questions_with_seller[questions_with_seller['is_abusing_seller'] == True]
normal_questions = questions_with_seller[questions_with_seller['is_abusing_seller'] == False]

print(f"\n어뷰징 판매자 리뷰: {len(abusing_reviews)}개")
print(f"정상 판매자 리뷰: {len(normal_reviews)}개")
print(f"어뷰징 판매자 질문: {len(abusing_questions)}개")
print(f"정상 판매자 질문: {len(normal_questions)}개")

### 13.1 리뷰 평점 분포 비교

In [ ]:
# 어뷰징 vs 정상 판매자의 리뷰 평점 분포 비교
abusing_rating_counts = abusing_reviews['review_rating'].value_counts().sort_index()
normal_rating_counts = normal_reviews['review_rating'].value_counts().sort_index()

# 비율로 변환
abusing_rating_pct = (abusing_rating_counts / len(abusing_reviews) * 100).round(1)
normal_rating_pct = (normal_rating_counts / len(normal_reviews) * 100).round(1)

fig = make_subplots(rows=1, cols=2, subplot_titles=('리뷰 평점 분포 (건수)', '리뷰 평점 분포 (비율 %)'))

# 건수 비교
fig.add_trace(go.Bar(x=normal_rating_counts.index.astype(str), y=normal_rating_counts.values, 
                     name='정상 판매자', marker_color='#00CC96'), row=1, col=1)
fig.add_trace(go.Bar(x=abusing_rating_counts.index.astype(str), y=abusing_rating_counts.values, 
                     name='어뷰징 판매자', marker_color='#EF553B'), row=1, col=1)

# 비율 비교
fig.add_trace(go.Bar(x=normal_rating_pct.index.astype(str), y=normal_rating_pct.values, 
                     name='정상 판매자', marker_color='#00CC96', showlegend=False), row=1, col=2)
fig.add_trace(go.Bar(x=abusing_rating_pct.index.astype(str), y=abusing_rating_pct.values, 
                     name='어뷰징 판매자', marker_color='#EF553B', showlegend=False), row=1, col=2)

fig.update_layout(
    height=450, 
    title_text="정상 vs 어뷰징 판매자: 리뷰 평점 분포 비교", 
    template='plotly_white',
    barmode='group'
)
fig.update_xaxes(title_text="평점", row=1, col=1)
fig.update_xaxes(title_text="평점", row=1, col=2)
fig.update_yaxes(title_text="리뷰 수", row=1, col=1)
fig.update_yaxes(title_text="비율 (%)", row=1, col=2)
fig.show()

# 평점별 통계 출력
print("\n=== 평점별 리뷰 비율 비교 ===")
print(f"{'평점':<6} {'정상 판매자':<15} {'어뷰징 판매자':<15}")
print("-" * 40)
for rating in range(1, 6):
    normal_pct = normal_rating_pct.get(rating, 0)
    abusing_pct = abusing_rating_pct.get(rating, 0)
    print(f"{rating}점     {normal_pct:>6.1f}%          {abusing_pct:>6.1f}%")

### 13.2 리뷰 텍스트 길이 및 내용 유무 비교

In [ ]:
# 리뷰 텍스트 길이 계산
reviews_with_seller['text_length'] = reviews_with_seller['review_text'].apply(
    lambda x: len(str(x)) if pd.notna(x) else 0
)
reviews_with_seller['has_text'] = reviews_with_seller['review_text'].notna()
reviews_with_seller['has_title'] = reviews_with_seller['review_title'].notna()

# 어뷰징/정상 분리
abusing_reviews = reviews_with_seller[reviews_with_seller['is_abusing_seller'] == True]
normal_reviews = reviews_with_seller[reviews_with_seller['is_abusing_seller'] == False]

fig = make_subplots(rows=1, cols=3, subplot_titles=(
    '리뷰 텍스트 길이 분포', 
    '리뷰 내용 유무 비율',
    '리뷰 제목 유무 비율'
))

# 텍스트 길이 분포 (박스플롯)
fig.add_trace(go.Box(y=normal_reviews['text_length'], name='정상', marker_color='#00CC96'), row=1, col=1)
fig.add_trace(go.Box(y=abusing_reviews['text_length'], name='어뷰징', marker_color='#EF553B'), row=1, col=1)

# 리뷰 내용 유무 비율
normal_has_text_pct = normal_reviews['has_text'].mean() * 100
abusing_has_text_pct = abusing_reviews['has_text'].mean() * 100
fig.add_trace(go.Bar(x=['정상', '어뷰징'], y=[normal_has_text_pct, abusing_has_text_pct], 
                     marker_color=['#00CC96', '#EF553B'], showlegend=False), row=1, col=2)

# 리뷰 제목 유무 비율
normal_has_title_pct = normal_reviews['has_title'].mean() * 100
abusing_has_title_pct = abusing_reviews['has_title'].mean() * 100
fig.add_trace(go.Bar(x=['정상', '어뷰징'], y=[normal_has_title_pct, abusing_has_title_pct], 
                     marker_color=['#00CC96', '#EF553B'], showlegend=False), row=1, col=3)

fig.update_layout(
    height=450, 
    title_text="정상 vs 어뷰징 판매자: 리뷰 텍스트 특성 비교", 
    template='plotly_white',
    showlegend=False
)
fig.update_yaxes(title_text="텍스트 길이 (자)", row=1, col=1)
fig.update_yaxes(title_text="내용 있는 비율 (%)", row=1, col=2)
fig.update_yaxes(title_text="제목 있는 비율 (%)", row=1, col=3)
fig.show()

# 통계 출력
print("\n=== 리뷰 텍스트 특성 비교 ===")
print(f"{'지표':<20} {'정상 판매자':<15} {'어뷰징 판매자':<15}")
print("-" * 55)
print(f"{'평균 텍스트 길이':<18} {normal_reviews['text_length'].mean():>8.1f}자       {abusing_reviews['text_length'].mean():>8.1f}자")
print(f"{'중앙값 텍스트 길이':<17} {normal_reviews['text_length'].median():>8.1f}자       {abusing_reviews['text_length'].median():>8.1f}자")
print(f"{'내용 있는 리뷰 비율':<16} {normal_has_text_pct:>8.1f}%        {abusing_has_text_pct:>8.1f}%")
print(f"{'제목 있는 리뷰 비율':<16} {normal_has_title_pct:>8.1f}%        {abusing_has_title_pct:>8.1f}%")

### 13.3 질문 및 답변 특성 비교

In [ ]:
# 질문/답변 특성 계산
questions_with_seller['has_answer'] = questions_with_seller['answer'].notna()
questions_with_seller['question_length'] = questions_with_seller['question'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
questions_with_seller['answer_length'] = questions_with_seller['answer'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)

# 어뷰징/정상 분리
abusing_questions = questions_with_seller[questions_with_seller['is_abusing_seller'] == True]
normal_questions = questions_with_seller[questions_with_seller['is_abusing_seller'] == False]

fig = make_subplots(rows=1, cols=3, subplot_titles=(
    '답변율', 
    '질문 길이 분포',
    '답변 길이 분포'
))

# 답변율 비교
normal_answer_rate = normal_questions['has_answer'].mean() * 100
abusing_answer_rate = abusing_questions['has_answer'].mean() * 100
fig.add_trace(go.Bar(x=['정상', '어뷰징'], y=[normal_answer_rate, abusing_answer_rate], 
                     marker_color=['#00CC96', '#EF553B'], showlegend=False), row=1, col=1)

# 질문 길이 분포
fig.add_trace(go.Box(y=normal_questions['question_length'], name='정상', marker_color='#00CC96'), row=1, col=2)
fig.add_trace(go.Box(y=abusing_questions['question_length'], name='어뷰징', marker_color='#EF553B'), row=1, col=2)

# 답변 길이 분포 (답변이 있는 것만)
normal_with_answer = normal_questions[normal_questions['has_answer']]
abusing_with_answer = abusing_questions[abusing_questions['has_answer']]
fig.add_trace(go.Box(y=normal_with_answer['answer_length'], name='정상', marker_color='#00CC96', showlegend=False), row=1, col=3)
fig.add_trace(go.Box(y=abusing_with_answer['answer_length'], name='어뷰징', marker_color='#EF553B', showlegend=False), row=1, col=3)

fig.update_layout(
    height=450, 
    title_text="정상 vs 어뷰징 판매자: 질문/답변 특성 비교", 
    template='plotly_white',
    showlegend=False
)
fig.update_yaxes(title_text="답변율 (%)", row=1, col=1)
fig.update_yaxes(title_text="질문 길이 (자)", row=1, col=2)
fig.update_yaxes(title_text="답변 길이 (자)", row=1, col=3)
fig.show()

# 통계 출력
print("\n=== 질문/답변 특성 비교 ===")
print(f"{'지표':<20} {'정상 판매자':<15} {'어뷰징 판매자':<15}")
print("-" * 55)
print(f"{'총 질문 수':<19} {len(normal_questions):>8}개        {len(abusing_questions):>8}개")
print(f"{'답변율':<21} {normal_answer_rate:>8.1f}%        {abusing_answer_rate:>8.1f}%")
print(f"{'평균 질문 길이':<18} {normal_questions['question_length'].mean():>8.1f}자       {abusing_questions['question_length'].mean():>8.1f}자")
print(f"{'평균 답변 길이':<18} {normal_with_answer['answer_length'].mean():>8.1f}자       {abusing_with_answer['answer_length'].mean():>8.1f}자")

### 13.4 상품당 리뷰/질문 수 비교

In [ ]:
# 상품별 리뷰/질문 수 집계
product_review_counts = reviews_with_seller.groupby(['product_id', 'is_abusing_seller']).size().reset_index(name='review_count')
product_question_counts = questions_with_seller.groupby(['product_id', 'is_abusing_seller']).size().reset_index(name='question_count')

# 어뷰징/정상 상품 분리
normal_product_reviews = product_review_counts[product_review_counts['is_abusing_seller'] == False]
abusing_product_reviews = product_review_counts[product_review_counts['is_abusing_seller'] == True]
normal_product_questions = product_question_counts[product_question_counts['is_abusing_seller'] == False]
abusing_product_questions = product_question_counts[product_question_counts['is_abusing_seller'] == True]

fig = make_subplots(rows=1, cols=2, subplot_titles=('상품당 리뷰 수', '상품당 질문 수'))

# 상품당 리뷰 수
fig.add_trace(go.Box(y=normal_product_reviews['review_count'], name='정상', marker_color='#00CC96'), row=1, col=1)
fig.add_trace(go.Box(y=abusing_product_reviews['review_count'], name='어뷰징', marker_color='#EF553B'), row=1, col=1)

# 상품당 질문 수
fig.add_trace(go.Box(y=normal_product_questions['question_count'], name='정상', marker_color='#00CC96', showlegend=False), row=1, col=2)
fig.add_trace(go.Box(y=abusing_product_questions['question_count'], name='어뷰징', marker_color='#EF553B', showlegend=False), row=1, col=2)

fig.update_layout(
    height=450, 
    title_text="정상 vs 어뷰징 판매자: 상품당 리뷰/질문 수 비교", 
    template='plotly_white',
    showlegend=False
)
fig.update_yaxes(title_text="리뷰 수", row=1, col=1)
fig.update_yaxes(title_text="질문 수", row=1, col=2)
fig.show()

# 통계 출력
print("\n=== 상품당 리뷰/질문 수 비교 ===")
print(f"{'지표':<22} {'정상 판매자':<15} {'어뷰징 판매자':<15}")
print("-" * 55)
print(f"{'상품당 평균 리뷰 수':<19} {normal_product_reviews['review_count'].mean():>8.1f}개       {abusing_product_reviews['review_count'].mean():>8.1f}개")
print(f"{'상품당 중앙값 리뷰 수':<18} {normal_product_reviews['review_count'].median():>8.1f}개       {abusing_product_reviews['review_count'].median():>8.1f}개")
print(f"{'상품당 평균 질문 수':<19} {normal_product_questions['question_count'].mean():>8.1f}개       {abusing_product_questions['question_count'].mean():>8.1f}개")
print(f"{'상품당 중앙값 질문 수':<18} {normal_product_questions['question_count'].median():>8.1f}개       {abusing_product_questions['question_count'].median():>8.1f}개")

### 13.5 종합 비교: 레이더 차트

In [ ]:
# 종합 비교를 위한 지표 계산 (정규화)
def normalize(value, min_val, max_val):
    if max_val == min_val:
        return 0.5
    return (value - min_val) / (max_val - min_val)

# 각 지표 계산
metrics = {
    '5점 리뷰 비율': (normal_rating_pct.get(5, 0), abusing_rating_pct.get(5, 0)),
    '평균 리뷰 길이': (normal_reviews['text_length'].mean(), abusing_reviews['text_length'].mean()),
    '리뷰 내용 작성율': (normal_has_text_pct, abusing_has_text_pct),
    '답변율': (normal_answer_rate, abusing_answer_rate),
    '평균 답변 길이': (normal_with_answer['answer_length'].mean(), abusing_with_answer['answer_length'].mean()),
    '상품당 리뷰 수': (normal_product_reviews['review_count'].mean(), abusing_product_reviews['review_count'].mean()),
}

# 정규화
categories = list(metrics.keys())
all_values = [v for vals in metrics.values() for v in vals]
min_val, max_val = min(all_values), max(all_values)

normal_values = []
abusing_values = []
for key in categories:
    normal_val, abusing_val = metrics[key]
    # 각 지표별로 정규화
    both_vals = [normal_val, abusing_val]
    min_v, max_v = min(both_vals), max(both_vals)
    if max_v > 0:
        normal_values.append(normal_val / max_v * 100)
        abusing_values.append(abusing_val / max_v * 100)
    else:
        normal_values.append(50)
        abusing_values.append(50)

# 레이더 차트
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=normal_values + [normal_values[0]],  # 닫힌 형태로
    theta=categories + [categories[0]],
    fill='toself',
    name='정상 판매자',
    line_color='#00CC96',
    fillcolor='rgba(0, 204, 150, 0.3)'
))

fig.add_trace(go.Scatterpolar(
    r=abusing_values + [abusing_values[0]],
    theta=categories + [categories[0]],
    fill='toself',
    name='어뷰징 판매자',
    line_color='#EF553B',
    fillcolor='rgba(239, 85, 59, 0.3)'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 100]
        )
    ),
    title='정상 vs 어뷰징 판매자: 리뷰/질문 특성 종합 비교',
    template='plotly_white',
    height=500
)
fig.show()

# 원본 값 출력
print("\n=== 지표별 원본 값 비교 ===")
print(f"{'지표':<18} {'정상 판매자':>15} {'어뷰징 판매자':>15}")
print("-" * 55)
for key in categories:
    normal_val, abusing_val = metrics[key]
    if '비율' in key or '율' in key:
        print(f"{key:<16} {normal_val:>12.1f}%    {abusing_val:>12.1f}%")
    elif '길이' in key:
        print(f"{key:<16} {normal_val:>12.1f}자   {abusing_val:>12.1f}자")
    else:
        print(f"{key:<16} {normal_val:>12.1f}개   {abusing_val:>12.1f}개")